## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [1]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = load_data()

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(200, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_train.shape
x_test = x_test.reshape(-1, 28, 28, 1) / 255
x_train.shape

model.fit(x_train, y_train, validation_data=(x_test, y_test))

1875/1875 [==============================] - 217s 114ms/step - loss: 0.1050 - accuracy: 0.9686 - val_loss: 0.0463 - val_accuracy: 0.9840


Afficher le nombre de paramètre du modèle

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 200)               1843400   
                                                                 
 dense_1 (Dense)             (None, 10)                2010      
                                                        

Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [3]:
model.save("model.h5")

import os

print("Taille du fichier: ",os.path.getsize("model.h5"), "octets" )

print("Taille occupée par parametre: ", os.path.getsize("model.h5")/model.count_params() , "octets" )

Taille du fichier:  22414000 octets
Taille occupée par parametre:  12.023220360621513 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)

Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [5]:
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

import os

print("Taille du fichier Keras: ",os.path.getsize("model.h5"), "octets" )
print("Taille du fichier Tensorflow Lite: ",os.path.getsize("model.tflite"), "octets" )

Taille du fichier Keras:  22414000 octets
Taille du fichier Tensorflow Lite:  7460112 octets


On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

converter_optimized = tf.lite.TFLiteConverter.from_keras_model(model)

converter_optimized.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_optimized = converter_optimized.convert()

with open('model_optimized.tflite', 'wb') as f:
    f.write(tflite_model_optimized)
  
import os
print("Taille du modèle optimisé:", os.path.getsize('model_optimized.tflite'), "octets")

Taille du modèle optimisé: 1870128 octets


Quelle type  de quantization Optimize.Default, utilise-t-elle ?


## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. 

Reprendre le modèle entraîné sur MNIST


In [24]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = load_data()

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(200, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_train.shape
x_test = x_test.reshape(-1, 28, 28, 1) / 255
x_train.shape

model.fit(x_train, y_train, validation_data=(x_test, y_test))

1875/1875 [==============================] - 197s 104ms/step - loss: 0.1097 - accuracy: 0.9659 - val_loss: 0.0374 - val_accuracy: 0.9873


A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [9]:
!pip install tensorflow-model-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 4.4 MB/s eta 0:00:00


In [10]:
import tensorflow_model_optimization as tfmot

qat_model = tfmot.quantization.keras.quantize_model(model)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Compiler le modèle

In [11]:
qat_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Afficher le summury du modèle. D'après vous ce modèle est-il quantifié ? 

In [12]:
qat_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 28, 28, 1)        3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 26, 26, 32)       387       
 erV2)                                                           
                                                                 
 quant_conv2d_1 (QuantizeWra  (None, 24, 24, 64)       18627     
 pperV2)                                                         
                                                                 
 quant_max_pooling2d (Quanti  (None, 12, 12, 64)       1         
 zeWrapperV2)                                                    
                                                                 
 quant_flatten (QuantizeWrap  (None, 9216)             1

Réentraîner votre modèle sur un sous ensemble des modèles sur une ou deux epochs et afficher la performance sur le train et test set

In [15]:
qat_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

qat_model.evaluate(x_test, y_test)

qat_model.evaluate(x_train, y_train)

Epoch 1/2
1875/1875 [==============================] - 243s 129ms/step - loss: 0.4104 - accuracy: 0.8805 - val_loss: 0.1010 - val_accuracy: 0.9678
Epoch 2/2
1875/1875 [==============================] - 270s 144ms/step - loss: 0.0824 - accuracy: 0.9754 - val_loss: 0.0569 - val_accuracy: 0.9808


Convertir votre modèle avec TFLite

In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)

converter_optimized.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_qat_model_optimized = converter_optimized.convert()

Comparer la performance du modèle Quantified aware training, au modèle original et au modèle quantifié post training

In [26]:
# Evaluation du modèle Quantified aware training
quantized_aware_train_loss, quantized_aware_train_accuracy = qat_model.evaluate(x_test, y_test)

# Evaluation du modèle original
original_loss, original_accuracy = model.evaluate(x_test, y_test)

# Evaluation du modèle quantifié post-training
quantized_post_train_loss, quantized_post_train_accuracy = tflite_qat_model_optimized.evaluate(x_test, y_test)



print("Performance du modèle original : loss = {:.4f}, accuracy = {:.4f}".format(original_loss, original_accuracy))
print("Performance du modèle quantifié post-training : loss = {:.4f}, accuracy = {:.4f}".format(quantized_post_train_loss, quantized_post_train_accuracy))
print("Performance du modèle Quantified aware training : loss = {:.4f}, accuracy = {:.4f}".format(quantized_aware_train_loss, quantized_aware_train_accuracy))

313/313 [==============================] - 7s 22ms/step - loss: 0.0374 - accuracy: 0.9873


AttributeError: ignored

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [ ]:
qat_model.save("qat_model.h5")

import os

print("Taille du modèle original : ", os.path.getsize('model.h5'))
print("Taille du modèle quantifié post-entraînement : ", os.path.getsize('quantized_model.tflite'))
print("Taille du modèle QAT : ", os.path.getsize('qat_model.h5'))

print("Taille du modèle original: ",os.path.getsize("model.h5"), "octets" )
print("Taille du modèle QAT: ",os.path.getsize("qat_model.h5"), "octets" )
print("Taille du modèle original: ",os.path.getsize("model.h5"), "octets" )

Bonus : déployer votre modèle sur votre téléphone ou un dispositif embarqué si vous en disposez d'un. 

Bonus : Obtenir un modèle qui sera à la fois quantifié et élagué (prunned) en s'aidant de la documentation (https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)

A l'aide de tensorflow lite / tensorflow lite micro 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>